In [1]:
import psycopg2
from psycopg2 import Error
import json
import gpxpy
import gpxpy.gpx
import time 
import datetime
import pandas as pd

### Connessione al DB

In [2]:
def create_dbconnection():
    with open('data.json') as json_file:
        data = json.load(json_file)
    try:
        connection = psycopg2.connect(user=data['user'],
                                      password=data['password'],
                                      host="127.0.0.1",
                                      port=data['port'],
                                      database=data['database'])
        return connection

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)

In [9]:
connection = create_dbconnection()
cursor = connection.cursor()

### Generali

Il sistema di coordinate è 4326 perchè non da problemi di visualizzazione in QGIS, ma ridà la ST_DISTANCE in gradi.

Il sistema di coordinate 3857 è usato nelle operazioni: ST_DISTANCE viene ridata in metri invece che in gradi

### Traiettorie personali gpx 

In [4]:
# nome della traiettoria
table_gpx = ['t'+str(x) for x in range(1,5)]
table_gpx

['t1', 't2', 't3', 't4']

In [5]:
# Create table TABLE_GPX
for table in table_gpx:
    query = """ DROP TABLE IF EXISTS """ + table + """ ;
                CREATE TABLE """ + table + """ (
                id serial, 
                LAT numeric, 
                LON numeric, 
                geom GEOMETRY(Point, 4326),
                time TIMESTAMP);"""
    cursor.execute(query)
    connection.commit()

In [6]:
for table in table_gpx:
   
    gpx_file = open('{}.gpx'.format(table) , 'r') 
    gpx = gpxpy.parse(gpx_file) 

    for track in gpx.tracks: 
        for segment in track.segments:
            id_point = 0
            for point in segment.points: 
                insert_query = """INSERT INTO """ + table + """(id,lat,lon,geom, time) 
                VALUES({},{},{},ST_GeomFromText('POINT({} {})', 4326),'{}');
                """.format(id_point, point.latitude, point.longitude, point.longitude, point.latitude, 
                           (point.time).strftime("%Y-%m-%d %H:%M:%S"))
                
                id_point += 1
                cursor.execute(insert_query)
                connection.commit()

### Data preprocessing - Map matching
Se si vuol fare una prova semplice si può creare ed utilizzare la tabella "part" per selezionare un sottoinsieme di punti su cui applicare il map matching (è un'operazione lunga, richiede un po' di tempo per applicarlo a tutti i punti della traccia)

In [10]:
# PT_ROADS: contiene punto e strada ad esso più vicino, con la distanza tra di loro. 
for table in table_gpx: 
    query = """
        drop table if exists {0}_pt_roads;

        create table {0}_pt_roads as
        select distinct on (pid)pid, Mi_id, p_geom, Mi_geom, distance
        from
            (select p.id as pid, p.geom as p_geom, Mi.id as Mi_id, 
             Mi.geom as Mi_geom, 
             ST_distance(
                 ST_transform(p.geom, 3857), 
                 ST_transform(ST_LineInterpolatePoint(Mi.geom, ST_LineLocatePoint(Mi.geom,p.geom)),3857)) as distance
             from {0} as p, stradevicine as Mi) as point_road_couple
        order by point_road_couple.pid, 
            st_distance(point_road_couple.p_geom, point_road_couple.Mi_geom);

        drop table if exists {0}_mapmatching;

        create table {0}_mapmatching as 
            (select pid, ST_LineInterpolatePoint(Mi_geom, ST_LineLocatePoint(Mi_geom,p_geom)), distance 
            from {0}_pt_roads
            where distance < 15 )
    """.format(table)

# MAP_MATCHING: contiene pid, geom del punto proiettato sulla strada pià vicino SE la distanza è < ...

    cursor.execute(query)
    connection.commit()

A questo punto in mapmatching ci ritroviamo ad avere solo i punti che sono stati modificati, dobbiamo creare una traiettoria unica con tutti i punti. 

Creiamo una tabella "final_points" dal left join di mapmatching e track_points, andiamo cioè a prendere tutti i punti di track_points e quelli che hanno lo stesso id nel mapmatching avranno delle colonne in più. 

In [ ]:
for table in table_gpx:
    query = """ drop table if exists {0}_final_points; 
                create table {0}_final_points as (
                        select * 
                        from {0}  
                        left join  {0}_mapmatching
                            on {0}.id = {0}_mapmatching.pid); 

                ALTER TABLE {0}_final_points
                    ADD final_geom geometry;
            """.format(table)
    cursor.execute(query)
    connection.commit()

In [ ]:
for table in table_gpx:
    # Selezione punti proiettati - con geom modificata espressa da st_lineinterpolarepoint
    query = "select st_lineinterpolatepoint, id from {0}_final_points where id = pid ".format(table)
    cursor.execute(query)
    risultati1 = cursor.fetchall()

    # Selezione punti rimasti uguali - hanno pid nulla dal leftjoin
    query = "select geom, id from {0}_final_points where pid is null".format(table)
    cursor.execute(query)
    risultati2 = cursor.fetchall()

    risultati = risultati1 + risultati2 

    for element in risultati: 
        query = "UPDATE {0}_final_points set final_geom = '{1}' where id = {2}".format(table,str(element[0]), str(element[-1]))
        cursor.execute(query)
        connection.commit()

In [ ]:
for table in table_gpx:
    query = """alter table {0}_final_points
        DROP COLUMN geom, DROP COLUMN pid,DROP COLUMN st_lineinterpolatepoint, DROP COLUMN distance, ADD time2 timestamp;
        UPDATE {0}_final_points SET time2 = time;
        alter table {0}_final_points DROP COLUMN time;   
        alter table {0}_final_points RENAME COLUMN time2 TO time;
        alter table {0}_final_points RENAME COLUMN final_geom to geom;
        """.format(table)
    cursor.execute(query)
    connection.commit()

### Creazione tabelle waypoint

In [ ]:
for TABLE in table_gpx:
    query = """ DROP TABLE IF EXISTS """ + TABLE +'_waypoint' + """ ;
                CREATE TABLE """ + TABLE +'_waypoint' + """ (
                id serial, 
                LAT numeric, 
                LON numeric, 
                geom GEOMETRY(Point, 4326),
                time TIMESTAMP
                );"""
    cursor.execute(query)
    connection.commit()


    id_point = 0

    gpx_file = open('{}.gpx'.format(TABLE) , 'r') 
    gpx = gpxpy.parse(gpx_file) 
 
    for point in gpx.waypoints: 
        insert_query = """INSERT INTO """ + TABLE +'_waypoint' + """(id,lat,lon,geom, time) 
                 VALUES({},{},{},ST_GeomFromText('POINT({} {})',4326),'{}');
                """.format(id_point, point.latitude, point.longitude, point.longitude, point.latitude, 
                           (point.time).strftime("%Y-%m-%d %H:%M:%S"))
        id_point += 1
        cursor.execute(insert_query)
        connection.commit()

### Stay point detection

In [ ]:
def delta_dist(x, y): # X,Y geom
    cur = connection.cursor()
    cur.execute("select st_distance(ST_TRANSFORM('{}', 3857),ST_TRANSFORM('{}', 3857));".format(str(x), str(y)))
    dd = cur.fetchall()
    return dd[0][0]

def delta_time(x, y): # X,Y timestamp
    cur = connection.cursor()
    dt = y-x
    return dt.total_seconds()

def calc_centroide(listapunti, table = TABLE): # X,Y id dei punti
    cur = connection.cursor()
    listapunti = tuple(listapunti)
    query = """ SELECT ST_CENTROID(st_union(t.geom)) as cent_point 
                FROM {} as t
                WHERE t.id in {} """.format(TABLE, listapunti) 
    cur.execute(query)
    centroide = cur.fetchall()
    geom_cent = centroide[0][0]
    return geom_cent

In [ ]:
def staypoint(distThre, timeThre):
    cur = connection.cursor()
    cur.execute("select distinct * from {} order by id".format(TABLE))
    data = cur.fetchall()
    
    query = """ ALTER TABLE {0} DROP COLUMN IF EXISTS id_centroide;     
                ALTER TABLE {0} ADD COLUMN id_centroide INTEGER """.format(TABLE)
    cursor.execute(query)
    connection.commit()
    
    i = 0
    num_points = len(data)
    id_centroide = 0
    stay_points = []
    
    while i < len(data):
        j = i+1
        
        while j < num_points:
            
            dist = delta_dist(data[i][3], data[j][3])
    
            if dist > distThre:
                temp = delta_time(data[i][4], data[j][4])
            
                if temp > timeThre:
                    lista_id = [i for i in range(i,j+1)]
                    query = """ UPDATE {0} SET id_centroide = {1}
                                WHERE id in {2}""".format(TABLE, id_centroide, tuple(lista_id))
                    cursor.execute(query)
                    connection.commit()
                    geom_cent = calc_centroide(lista_id)
                    time_inizio = data[i][4]
                    time_fine = data[j][4]
                    point = [id_centroide, geom_cent, temp, time_inizio, time_fine]
                    stay_points.append(point)
                    id_centroide += 1
                    
                break
            j += 1
        i = j
        
 
   
    print('\nSono stati identificati {} staypoints con sogliadist < {}m e sogliatime > {}s: \n'.format(len(stay_points), distThre, timeThre)) 
    cur.execute(""" DROP TABLE if exists {2}_staypoints_{0}m_{1}s; 
                CREATE TABLE {2}_staypoints_{0}m_{1}s (id varchar, geom geometry, 
                deltat numeric, time_inizio timestamp, time_fine timestamp)""".format(distThre, timeThre, TRACCIA))
    connection.commit()
    
    
    for point in stay_points:
        print('staypoint {}: intervallo {}'.format(point[0], point[2]/60))
        cur.execute(""" INSERT INTO {0}_staypoints_{1}m_{2}s
                    VALUES ({3}, '{4}', {5}, '{6}', '{7}')
                    """.format(TRACCIA, distThre, timeThre, point[0], point[1], point[2], point[3], point[4]))
        connection.commit()
    print("END TRACCIA")

In [ ]:
metri = 50
secondi = 60*3

for t in table_gpx: 
    TRACCIA = t
    TABLE = str(t) + "_final_points"
    sp = staypoint(metri, secondi)

### Risultati staypoint: 
- Detected waypoints
- Distanza media tra waypoint e staypoint
- tot wp / tot sp

In [ ]:
total_wp = 0
total_sp = 0
total_detected = 0
total_far_sp = 0
for t in table_gpx:
    TABLE_STAYPOINTS = t + '_staypoints_' + str(metri) + 'm_' + str(secondi) + 's'
    TABLE_WAYPOINTS = t + "_waypoint"
    
    cursor.execute("""ALTER TABLE {0} DROP COLUMN IF EXISTS detected_by_sp;
                     ALTER TABLE {0} ADD COLUMN detected_by_sp INTEGER;
                     
                     ALTER TABLE {1} DROP COLUMN IF EXISTS near_wp;
                     ALTER TABLE {1} ADD COLUMN near_wp INTEGER;""".format(TABLE_WAYPOINTS, TABLE_STAYPOINTS))
    connection.commit() #aggiungiamo una colonna detected alle tabelle di waypoint
    
    #Selezioniamo wp vicini a sp
    query = """select wp.id as wpid, sp.id as spid, st_distance(st_transform(wp.geom, 3857),st_transform(sp.geom, 3857))
                from {0} as wp, {1} as sp
                where st_distance(
                        st_transform(wp.geom, 3857),
                        st_transform(sp.geom, 3857)
                ) < 30""".format(TABLE_WAYPOINTS, TABLE_STAYPOINTS)
    cursor.execute(query)
    wp_vicini_sp = cursor.fetchall()
    
    for row in wp_vicini_sp: #Aggiorniamo la tabella di wp, indice ci dice se il punto è stato rilevato
        cursor.execute(""" UPDATE {0} SET detected_by_sp = 1 WHERE {0}.id = '{1}'""".format(TABLE_WAYPOINTS, str(row[0])))
        connection.commit()
        cursor.execute(""" UPDATE {0} SET near_wp = 1 WHERE {0}.id = '{1}'""".format(TABLE_STAYPOINTS, row[1]))
        connection.commit()
    
    cursor.execute("select count(detected_by_sp) from {}".format(TABLE_WAYPOINTS))
    track_det = cursor.fetchall()[0][0]
    total_detected += track_det
    
    cursor.execute("select count(*) from {}".format(TABLE_WAYPOINTS))
    track_wp = cursor.fetchall()[0][0]
    total_wp += track_wp
    
    cursor.execute("select count(*) from {}".format(TABLE_STAYPOINTS))
    track_sp = cursor.fetchall()[0][0]
    total_sp += track_sp
    
    cursor.execute("select COUNT(*)-count(near_wp) from {}".format(TABLE_STAYPOINTS))
    track_far_sp = cursor.fetchall()[0][0]
    total_far_sp += track_far_sp

    print(t, "detected_by_sp:", round(track_det/track_wp, 2), "perc sp lontani:",round(track_far_sp/track_sp,2))

In [ ]:
print("Risultati di staypoint {}m, {}s: ".format(metri,secondi))
print("Percentuale di WP rilevati: ", round(total_detected/total_wp,3))
print("Percentuale di staypoint lontati da wp: ", round(total_far_sp/total_sp,3))


### DBSCAN

CLUS_INDEX:

    if 0: noise 

    if > 0: appartenenza al cluster di id clus_index


In [ ]:
def get_vicini(id_p, eps, tempo, deltat):
    
    if tempo:    
        cursor.execute("""  SELECT t2.id FROM {0}_final_points as t1, {0}_final_points as t2
                        WHERE st_distance(st_transform(t1.geom, 3857), st_transform(t2.geom, 3857)) <= {1} and t1.id = '{2}' 
                        AND abs(extract(epoch from t2.time::timestamp - t1.time::timestamp )) <{3}
                        """.format(TABLE, eps, str(id_p), deltat))
    else:
        cursor.execute(""" SELECT t2.id FROM {0}_final_points as t1, {0}_final_points as t2
                        WHERE st_distance(st_transform(t1.geom, 3857), st_transform(t2.geom, 3857)) <= {1} 
                        AND t1.id = '{2}' """.format(TABLE, eps, id_p))
    
    data = cursor.fetchall()
    vicini = []
    for i in range(len(data)):
        vicini.append(int(data[i][0]))
        
    return vicini
    
def espansione_cluster(p, vicinato, cluster_index, eps, minpts, labels, tempo, deltat):
    seed = vicinato
    for nodo in seed:
        if labels[nodo] == -1 or labels[nodo] == 0: # non visitato o noise
            labels[nodo] = cluster_index            
            vicini_nodo = get_vicini(nodo, eps, tempo, deltat)
            if len(vicini_nodo) >= minpts:
                seed.extend(vicini_nodo)    

In [ ]:
def DBSCAN(eps, minpts, tempo = False, deltat = 0):

    cursor.execute("select count(*) from {}_final_points".format(TABLE))
    dataset_size = cursor.fetchone()[0]
    
    labels = [-1 for i in range(dataset_size+1)] # -1 è nodo non visitato
    cluster_index = 1
    
    for i, lbl in enumerate(labels):
        if lbl == -1:
            vicinato = get_vicini(i, eps, tempo, deltat)
            if len(vicinato) >= minpts:
                labels[i] = cluster_index
                espansione_cluster(i, vicinato, cluster_index, eps, minpts, labels, tempo, deltat)
                cluster_index += 1 
            else:
                labels[i] = 0 # 0 nodo rumore
    
    new_column = 'clus_index_tempo' if tempo else 'clus_index'
    cursor.execute(""" ALTER TABLE {0}_final_points DROP COLUMN IF EXISTS {1};     
                    ALTER TABLE {0}_final_points ADD COLUMN {1} INTEGER """.format(TABLE, new_column))
    connection.commit()
    
    for i in range(len(labels)):
        cursor.execute(""" UPDATE {0}_final_points SET {1} = {2} 
                        WHERE {0}_final_points.id = '{3}'""".format(TABLE, new_column, labels[i], i))
        connection.commit()
        
    print('Numero cluster: ' + str(cluster_index-1))


In [ ]:
PATH = 'D:/Unimi/Geospatial data mng/Progetto/immagini/'
for t in table_gpx:
    TABLE = t
    DBSCAN(20, 5)
    DBSCAN(20, 5, tempo = True, deltat = 60*5)
    print('\n')

### Calcolo punti di sosta per DBSCAN

In [ ]:
for TABLE in table_gpx:
    cursor.execute("select distinct clus_index_tempo from {}_final_points".format(TABLE))
    data = cursor.fetchall()
    
    cursor.execute(""" DROP TABLE if exists {0}_DBSCAN_centroidi; 
                CREATE TABLE {0}_DBSCAN_centroidi (id varchar, geom geometry)""".format(TABLE))
    connection.commit()

    for clus_id in data:
        if clus_id[0] > 0:
            cursor.execute("select distinct id from {}_final_points where clus_index_tempo = {}".format(TABLE,clus_id[0] ))
            risultati = cursor.fetchall()
            lista_punti_cluster = [elem[0] for elem in risultati]
            geom_centroide = calc_centroide(lista_punti_cluster, table = str(TABLE)+'_final_points')
            cursor.execute("""INSERT INTO {0}_DBSCAN_centroidi
                            VALUES({1}, '{2}')""".format(TABLE, int(clus_id[0]), str(geom_centroide)))
            connection.commit()
    

### Risultati di DBSCAN

In [ ]:
distanze = 0
counter_distanze = 0
total_wp = 0
total_fermate_dbscan = 0
total_detected = 0
total_far_fermate = 0
for t in table_gpx:
    TABLE_FERMATE_DBSCAN = t + '_dbscan_centroidi'
    TABLE_WAYPOINTS = t + "_waypoint"
    
    cursor.execute("""ALTER TABLE {0} DROP COLUMN IF EXISTS detected_by_dbscan;
                     ALTER TABLE {0} ADD COLUMN detected_by_dbscan INTEGER;
                     
                     ALTER TABLE {1} DROP COLUMN IF EXISTS near_wp;
                     ALTER TABLE {1} ADD COLUMN near_wp INTEGER;""".format(TABLE_WAYPOINTS, TABLE_FERMATE_DBSCAN))
    connection.commit() #aggiungiamo una colonna detected alle tabelle di waypoint
    
    #Selezioniamo wp vicini a CENTROIDI 
    query = """select wp.id as wpid, DB.id as DBid, st_distance(st_transform(wp.geom, 3857),st_transform(DB.geom, 3857))
                from {0} as wp, {1} as DB
                where st_distance(
                        st_transform(wp.geom, 3857),
                        st_transform(DB.geom, 3857)
                ) < 30""".format(TABLE_WAYPOINTS, TABLE_FERMATE_DBSCAN)
    cursor.execute(query)
    wp_vicini_db = cursor.fetchall()
    
    
    for row in wp_vicini_db: #Aggiorniamo la tabella di wp, indice ci dice se il punto è stato rilevato
        cursor.execute(""" UPDATE {0} SET detected_by_dbscan = 1 WHERE {0}.id = '{1}'""".format(TABLE_WAYPOINTS, str(row[0])))
        connection.commit()
        cursor.execute(""" UPDATE {0} SET near_wp = 1 WHERE {0}.id = '{1}'""".format(TABLE_FERMATE_DBSCAN, row[1]))
        connection.commit()
        distanze += row[2]
    counter_distanze += len(wp_vicini_db)
    
    cursor.execute("select count(detected_by_dbscan) from {}".format(TABLE_WAYPOINTS))
    track_det = cursor.fetchall()[0][0]
    total_detected += track_det
    
    cursor.execute("select count(*) from {}".format(TABLE_WAYPOINTS))
    track_wp = cursor.fetchall()[0][0]
    total_wp += track_wp
    
    cursor.execute("select count(*) from {}".format(TABLE_FERMATE_DBSCAN))
    track_fermate_dbscan = cursor.fetchall()[0][0]
    total_fermate_dbscan += track_fermate_dbscan
    
    cursor.execute("select COUNT(*)-count(near_wp) from {}".format(TABLE_FERMATE_DBSCAN))
    track_far_fermata = cursor.fetchall()[0][0]
    total_far_fermate += track_far_fermata
    
    print(t, "detected_by_dbscan:", round(track_det/track_wp, 2),
          "perc centroidi lontani:",round(track_far_fermata/track_fermate_dbscan,2))
    

In [ ]:
print("Risultati dbscan temporale:")
print("Percentuale di WP rilevati: ", total_detected/total_wp)
print("Percentuale di staypoint lontati da wp: ", total_far_fermate/total_fermate_dbscan)


### Visualizzazione

In [ ]:
# from mpl_toolkits import mplot3d
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# def plot3D (path = PATH, tempo = False, complete = False):
    
#     fig = plt.figure(figsize=(26, 20)) 
#     ax = Axes3D(fig)
#     title = 'cluster_tempo'if tempo else 'cluster'
#     plt.title(title, size = 50, pad = 60, loc = 'right')
#     ax.set_xlabel("longitude", size=25, labelpad=30) 
#     ax.set_ylabel("latitude", size=25, labelpad=30) 
#     ax.set_zlabel("time", size=25, labelpad=30)
    
#     query = """ SELECT *, extract(epoch from time::timestamp) 
#                 FROM {}_final_points ORDER BY time""".format(TABLE)
#     cursor.execute(query)
#     data = cursor.fetchall()
#     lons = [point[2] for point in data] 
#     lats = [point[1] for point in data] 
#     times = [point[-1] for point in data]
    
#     index = -2 if tempo else -3 
#     clus_index = [point[index] for point in data] # -2 con tempo, -3 cluster normali
#     ax.scatter(lons, lats, times, c = clus_index, cmap="plasma", s= 95, alpha = 1)
    
#     if complete:
#         title = title + 'complete' 
#         query2 = """ SELECT extract(epoch from time_inizio) as time, st_y(st_centroid(geom)) as lat,
#                      st_x(st_centroid(geom)) as long FROM {} ORDER BY time""".format(TABLE_SP)
#         cursor.execute(query2)
#         data2 = cursor.fetchall()
#         lons2 = [point[2] for point in data2] 
#         lats2 = [point[1] for point in data2] 
#         times2 = [point[0] for point in data2] 
#         ax.scatter(lons2, lats2, times2, c='black', s= 600, marker = 'X', alpha = 1) 
        
#         query3 = """ SELECT st_y(st_centroid(geom)) as lat, st_x(st_centroid(geom)) as long, 
#                      extract(epoch from time::timestamp) as time FROM {}_waypoint""".format(TABLE)
#         cursor.execute(query3)
#         data3 = cursor.fetchall()
#         lons3 = [point[1] for point in data3] 
#         lats3 = [point[0] for point in data3] 
#         times3 = [point[2] for point in data3]
#         ax.scatter(lons3, lats3, times3, c='red', s= 600, marker = 'X', alpha = 1)
        
#     plt.savefig(path + TABLE + '_' + title)
#     plt.show()


In [ ]:
# PATH = 'C:/Users/user/Desktop/ProgettoGDM/Immagini/{}/'.format(TABLE)
 
# metri = 50
# secondi = 180
# for TABLE in table_gpx:
#     TABLE_SP = TABLE + '_staypoints_{}m_{}s'.format(metri, secondi)
#     plot3D(tempo = False)
#     plot3D(tempo = True)
#     plot3D(tempo = True, complete = True)

---------------------------------------------------------------

### Modo per leggere i file csv in modo tale che siano visibili in qgis senza problemi

In [ ]:
query = """ drop table if exists stradevicine;
            create table stradevicine (
            id numeric, 
            geom GEOMETRY(Linestring, 4326),
            nome varchar(100));"""
cursor.execute(query)
connection.commit()

with open('strade.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    line_count = 0
    for row in csv_reader: 
        if line_count == 0: 
            print("columns: " + row[0], row[1], row[2])
            line_count += 1
        else:
            if "'" in row[2]:
                query = """INSERT INTO stradevicine(id,geom,nome) 
                    VALUES({},'{}', NULL)
                """.format(row[0], row[1])
            else: 
                query = """INSERT INTO stradevicine(id,geom,nome) 
                    VALUES({},'{}', '{}')
                """.format(row[0], row[1], row[2])
            line_count  += 1
            cursor.execute(query)
            connection.commit()

### Modo per creare file csv

In [ ]:
import pandas as pd

In [ ]:
query = "select * from stradevicine"
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=['id', 'geom', 'name'])
df.to_csv("stradevicine.csv", index=False)

In [ ]:
query = "select * from strademilanobollate"
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=['id', 'geom', 'name'])
df.to_csv("strademilanobollate.csv", index=False)

---------------------------------------------------------------

### Lettura dataset pechino

In [ ]:
USER = '000'
TABLE_GEOLIFE = 'traiettoria_pechino_' + USER

In [ ]:
query = """ DROP TABLE IF EXISTS """ + TABLE_GEOLIFE + """ ;
            CREATE TABLE """ + TABLE_GEOLIFE + """ (
            id serial, 
            LAT numeric, 
            LON numeric, 
            geom GEOMETRY(Point, 4326),
            time TIMESTAMP);"""
cursor.execute(query)
connection.commit()

In [ ]:
with open('20081024041230.csv') as csv_file:
    df = pd.read_csv(csv_file, usecols = [0, 1, 5, 6], names = ['lat', 'long', 'data', 'hour'], skiprows = 6)

total_points = df.shape[0]  

for i in range(total_points):
    point = df.loc[i,:] 
    str_timestamp = point['data'] +' ' + point['hour']
    query = """ INSERT INTO """ + TABLE_GEOLIFE + """(id, lat, lon, geom, time)
                VALUES({}, {}, {}, ST_GeomFromText('POINT({} {})', 4326),'{}') 
            """.format(i, point['lat'], point['long'], point['long'], point['lat'], str_timestamp)
    
    cursor.execute(query)
    connection.commit()

---------------------------------------------------------------

In [ ]:
# NB: scegliere questa da eseguire per il map matching durante un'eventuale demo per l'esame! 

query = """
    drop table if exists part;
    
    create table part as 
    select * from track_points where id > 240 and id < 280;
  
    drop table if exists pt_roadsPROVA;
    
    create table pt_roadsPROVA as
    select distinct on (pid)pid, Mi_id, p_geom, Mi_geom, distance
    from
        (select part.id as pid, part.geom as p_geom, Mi.id as Mi_id, 
         Mi.geom as Mi_geom, 
         ST_distance(
             ST_transform(part.geom, 3857), 
             ST_transform(ST_LineInterpolatePoint(Mi.geom, ST_LineLocatePoint(Mi.geom,part.geom)),3857)) as distance
         from part, "LinestringRouteMilan" as Mi) as point_road_couple
    order by point_road_couple.pid, 
        st_distance(point_road_couple.p_geom, point_road_couple.Mi_geom);
    
    create table mapmatchingPROVA as 
        (select pid, ST_LineInterpolatePoint(Mi_geom, ST_LineLocatePoint(Mi_geom,p_geom)), distance 
        from pt_roads
        )
"""

In [ ]:
# WAYPOINT = punti aggiuti manualmente - aggiungere dopo e vedere se coincidono con stay point

# for point in gpx.waypoints: 
#     insert_query = """INSERT INTO """ + TABLE_GPX + """(traccia,lat,lon,geom, time) 
#             VALUES({},{},{},ST_GeomFromText('POINT({} {})',3857),'{}', '{}');
#             """.format(i, point.latitude, point.longitude, point.latitude, point.longitude, 
#                        (point.time).strftime("%Y-%m-%d %H:%M:%S"), point.name)

#     cursor.execute(insert_query)
#     connection.commit()